# How to access FPL API data

In [1]:
import requests, os, time
from pprint import pprint
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', None)

## Bootstrap_url
This endpoint returns the data for players, positions and teams. We can inspect the different entities returned by this endpoint using the `pprint()` function with the `depth` parameter set to 1

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11436072}


### Players
Player data can be found in the `elements` entity. The response from the `bootstrap_url` endpoint is in json format, which is essentially just a dictionary of keys and values.   
Therefore, to get the `elements` entity, we can use `r['elements']` 

In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 8,
 'bonus': 13,
 'bps': 581,
 'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'clean_sheets': 12,
 'clean_sheets_per_90': 0.38,
 'code': 84450,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -2,
 'cost_change_start_fall': 2,
 'creativity': '671.7',
 'creativity_rank': 29,
 'creativity_rank_type': 22,
 'direct_freekicks_order': 2,
 'direct_freekicks_text': '',
 'dreamteam_count': 1,
 'element_type': 3,
 'ep_next': '4.1',
 'ep_this': '4.1',
 'event_points': 1,
 'expected_assists': '3.86',
 'expected_assists_per_90': 0.12,
 'expected_goal_involvements': '7.63',
 'expected_goal_involvements_per_90': 0.24,
 'expected_goals': '3.77',
 'expected_goals_conceded': '35.91',
 'expected_goals_conceded_per_90': 1.13,
 'expected_goals_per_90': 0.12,
 'first_name': 'Granit',
 'form': '3.6',
 'form_rank': 65,
 'form_rank_type': 35,
 'goals_co

### Use pandas for cleaner visualisation
Pandas can easily convert JSON data into a dataframe using the `json_normalize()` function.

In [4]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,3,Xhaka,1,3
1,4,Elneny,1,3
2,5,Holding,1,2
3,6,Partey,1,3
4,7,Ødegaard,1,3


### Teams

In [5]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1245,1285,1250,1250,1240,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1070,1100,1070,1075,1070,1130,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,2,None,False,0,1035,1095,1020,1110,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1115,1180,1100,1160,1130,1200,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1170,1175,1140,1150,1200,1200,131


### Player positions

In [6]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],260
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],330
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],92


## Combined dataset

In [7]:
# select columns of interest from players df
df = players[[
    'id', 'first_name', 'second_name', 'web_name', 'team', 'element_type']]

# join team name
df = df.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(
    ['team', 'id'], axis=1
# join player positions
).merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(
    ['element_type', 'id'], axis=1)

df.rename(columns={'id_player': 'id', 'name': 'team_name',
                   'singular_name_short': 'position'},
          inplace=True)

df.head()

,id,first_name,second_name,web_name,team_name,position
0,3,Granit,Xhaka,Xhaka,Arsenal,MID
1,4,Mohamed,Elneny,Elneny,Arsenal,MID
2,6,Thomas,Partey,Partey,Arsenal,MID
3,7,Martin,Ødegaard,Ødegaard,Arsenal,MID
4,9,Nicolas,Pépé,Pépé,Arsenal,MID


### Player gameweek history
Individual gameweek scores can be retrieved on a per-player basis

In [8]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [9]:
pd.json_normalize(r['history']).head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,4,1,7,0,False,2022-08-05T19:00:00Z,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,45,0,86132,0,0
1,4,11,10,0,True,2022-08-13T14:00:00Z,4,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,45,-12599,87552,5829,18428
2,4,21,3,0,False,2022-08-20T16:30:00Z,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,44,21882,131934,57920,36038
3,4,31,9,2,True,2022-08-27T16:30:00Z,2,1,4,90,0,0,0,1,0,0,0,0,0,0,0,15,4.2,3.8,0.0,0.8,1,0.00,0.04,0.04,0.83,44,-18950,115883,14683,33633
4,4,41,2,0,True,2022-08-31T18:30:00Z,2,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,44,-9174,110818,10920,20094


If we create a function to get gameweek history for a player, we can then apply this function across all the rows of the `players` dataframe

In [10]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])

    # avoid getting rate limited
    time.sleep(.3)
    
    return df

    
# show player #4's gameweek history
get_gameweek_history(player_id=4)[
    ['round', 'total_points', 'minutes', 'goals_scored', 'assists']].head()

,round,total_points,minutes,goals_scored,assists
0,1,0,0,0,0
1,2,0,0,0,0
2,3,0,0,0,0
3,4,2,90,0,0
4,5,0,0,0,0


The same can be done to get past seasons' summaries

In [11]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
        base_url + 'element-summary/' + str(player_id) + '/').json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    df.insert(0, 'id', player_id)

    # avoid getting rate limited
    time.sleep(.3)
    
    return df


# show player #1's gameweek history
get_season_history(player_id=1)[
    ['id', 'season_name', 'total_points', 'minutes',
     'goals_scored', 'assists']].head(10)

,id,season_name,total_points,minutes,goals_scored,assists
0,1,2015/16,86,1965,0,2
1,1,2016/17,102,2515,0,3
2,1,2017/18,85,2794,0,3
3,1,2018/19,52,1493,1,2
4,1,2019/20,61,1553,1,1
5,1,2020/21,28,744,0,1
6,1,2021/22,48,1481,1,1
